In [1]:
pip install pyautogui

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


gesture-controlled virtual mouse system

In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    model_complexity=0,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

screen_width, screen_height = pyautogui.size()

# Variables
prev_x, prev_y = 0, 0
smoothening = 5
mouse_active = False
click_performed = False
right_click_performed = False
double_click_performed = False  # New flag for double click

# Function to detect which fingers are up
def fingers_up(hand_landmarks):
    fingers = []
    tip_ids = [4, 8, 12, 16, 20]

    # Thumb
    if hand_landmarks.landmark[tip_ids[0]].x < hand_landmarks.landmark[tip_ids[0] - 1].x:
        fingers.append(1)
    else:
        fingers.append(0)

    # Other four fingers
    for id in range(1, 5):
        if hand_landmarks.landmark[tip_ids[id]].y < hand_landmarks.landmark[tip_ids[id] - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)
    return fingers

# Calculate distance between two landmarks
def distance(p1, p2):
    return np.linalg.norm(np.array([p1.x, p1.y]) - np.array([p2.x, p2.y]))

while True:
    success, frame = cap.read()
    if not success:
        continue

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            lm_list = hand_landmarks.landmark
            fingers = fingers_up(hand_landmarks)

            # Spiderman Gesture to Toggle Mouse Control
            if fingers[0] == 1 and fingers[1] == 1 and fingers[4] == 1 and fingers[2] == 0 and fingers[3] == 0:
                mouse_active = not mouse_active
                time.sleep(1)

            if mouse_active:
                # Track Index Finger tip (landmark 8)
                x = int(lm_list[8].x * 640)
                y = int(lm_list[8].y * 480)

                screen_x = np.interp(x, (0, 640), (0, screen_width))
                screen_y = np.interp(y, (0, 480), (0, screen_height))

                # Smooth cursor movement
                curr_x = prev_x + (screen_x - prev_x) / smoothening
                curr_y = prev_y + (screen_y - prev_y) / smoothening
                pyautogui.moveTo(curr_x, curr_y)
                prev_x, prev_y = curr_x, curr_y

                # Distance calculations
                dist_index_thumb = distance(lm_list[4], lm_list[8])
                dist_middle_thumb = distance(lm_list[4], lm_list[12])
                dist_index_middle = distance(lm_list[8], lm_list[12])

                # Left Click (Thumb + Index Finger Close)
                if dist_index_thumb < 0.03:
                    if not click_performed:
                        pyautogui.click()
                        click_performed = True
                else:
                    click_performed = False

                # Right Click (Thumb + Middle Finger Close)
                if dist_middle_thumb < 0.03:
                    if not right_click_performed:
                        pyautogui.click(button='right')
                        right_click_performed = True
                else:
                    right_click_performed = False

                # Double Click (Thumb + Index + Middle Close)
                if dist_index_thumb < 0.03 and dist_middle_thumb < 0.03 and dist_index_middle < 0.03:
                    if not double_click_performed:
                        pyautogui.doubleClick()
                        double_click_performed = True
                else:
                    double_click_performed = False

                # Scroll Up (Index + Middle up)
                if fingers == [0, 1, 1, 0, 0]:
                    pyautogui.scroll(30)
                    time.sleep(0.2)

                # Scroll Down (Index + Middle + Ring up)
                if fingers == [0, 1, 1, 1, 0]:
                    pyautogui.scroll(-30)
                    time.sleep(0.2)

    # Show Webcam Feed
    cv2.putText(frame, f'Mouse Active: {mouse_active}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 255, 0) if mouse_active else (0, 0, 255), 2)
    cv2.imshow("Virtual Mouse", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC key to exit
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 

In [1]:
!pip install mediapipe


[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install pycaw screen-brightness-control comtypes



   ---------------- ----------------------- 2/5 [comtypes]
   ---------------- ----------------------- 2/5 [comtypes]
   ---------------- ----------------------- 2/5 [comtypes]
   ---------------- ----------------------- 2/5 [comtypes]
   ---------------- ----------------------- 2/5 [comtypes]
   ---------------- ----------------------- 2/5 [comtypes]
   ---------------- ----------------------- 2/5 [comtypes]
   ---------------- ----------------------- 2/5 [comtypes]
   ---------------- ----------------------- 2/5 [comtypes]
   ------------------------ --------------- 3/5 [screen-brightness-control]
   -------------------------------- ------- 4/5 [pycaw]
   ---------------------------------------- 5/5 [pycaw]

Note: you may need to restart the kernel to use updated packages.


gesture-based brightness and volume control system

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import screen_brightness_control as sbc
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from comtypes import CLSCTX_ALL
from ctypes import cast, POINTER
import collections
import time

# Initialize mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize audio control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Helper to get angle
def get_signed_angle(finger_tip, finger_base):
    vector = np.array([finger_tip[0] - finger_base[0], finger_tip[1] - finger_base[1]])
    norm = np.linalg.norm(vector)
    if norm == 0:
        return 0
    vector = vector / norm
    ref_vector = np.array([0, -1])
    angle_rad = np.arccos(np.clip(np.dot(vector, ref_vector), -1.0, 1.0))
    angle_deg = np.degrees(angle_rad)
    cross = np.cross(ref_vector, vector)
    if cross < 0:
        angle_deg = -angle_deg
    return angle_deg

# Helper to detect spiderman gesture
def is_spiderman(hand_landmarks):
    fingers = []
    tip_ids = [mp_hands.HandLandmark.THUMB_TIP,
               mp_hands.HandLandmark.INDEX_FINGER_TIP,
               mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
               mp_hands.HandLandmark.RING_FINGER_TIP,
               mp_hands.HandLandmark.PINKY_TIP]

    dip_ids = [mp_hands.HandLandmark.THUMB_IP,
               mp_hands.HandLandmark.INDEX_FINGER_PIP,
               mp_hands.HandLandmark.MIDDLE_FINGER_PIP,
               mp_hands.HandLandmark.RING_FINGER_PIP,
               mp_hands.HandLandmark.PINKY_PIP]

    for tip_id, dip_id in zip(tip_ids, dip_ids):
        tip_y = hand_landmarks.landmark[tip_id].y
        dip_y = hand_landmarks.landmark[dip_id].y
        fingers.append(tip_y < dip_y)

    if fingers[1] and not fingers[2] and not fingers[3] and fingers[4]:
        return True
    return False

# Track previous angles
prev_right_angle = None
prev_left_angle = None

# Buffers to smooth
angle_buffer_size = 3
right_angle_buffer = collections.deque(maxlen=angle_buffer_size)
left_angle_buffer = collections.deque(maxlen=angle_buffer_size)

# Cached brightness and volume
try:
    cached_brightness = sbc.get_brightness(display=0)[0]
except:
    cached_brightness = 50
last_brightness_update = time.time()

# Activation flags
right_active = False
left_active = False
last_right_toggle = 0
last_left_toggle = 0
toggle_cooldown = 1  # seconds

# Control flags
brightness_control = False
volume_control = False
control_display_duration = 1.5  # seconds
last_brightness_control_time = 0
last_volume_control_time = 0

# Main loop
while True:
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    # Get current volume level
    current_volume = volume.GetMasterVolumeLevelScalar()

    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            hand_label = handedness.classification[0].label

            h, w, _ = frame.shape
            middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
            middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

            middle_mcp = (int(middle_mcp.x * w), int(middle_mcp.y * h))
            middle_tip = (int(middle_tip.x * w), int(middle_tip.y * h))

            angle = get_signed_angle(middle_tip, middle_mcp)

            if is_spiderman(hand_landmarks):
                if hand_label == 'Right' and (time.time() - last_right_toggle) > toggle_cooldown:
                    right_active = not right_active
                    last_right_toggle = time.time()
                    print(f"Right hand active: {right_active}")

                if hand_label == 'Left' and (time.time() - last_left_toggle) > toggle_cooldown:
                    left_active = not left_active
                    last_left_toggle = time.time()
                    print(f"Left hand active: {left_active}")

            if hand_label == 'Right' and right_active:
                right_angle_buffer.append(angle)
                smooth_angle = np.mean(right_angle_buffer)

                if prev_right_angle is not None:
                    angle_diff = smooth_angle - prev_right_angle

                    if abs(angle_diff) > 1:
                        cached_brightness += angle_diff * 0.4
                        cached_brightness = np.clip(cached_brightness, 0, 100)
                        brightness_control = True
                        last_brightness_control_time = time.time()

                prev_right_angle = smooth_angle

            elif hand_label == 'Left' and left_active:
                left_angle_buffer.append(angle)
                smooth_angle = np.mean(left_angle_buffer)

                if prev_left_angle is not None:
                    angle_diff = smooth_angle - prev_left_angle

                    if abs(angle_diff) > 1:
                        current_volume = volume.GetMasterVolumeLevelScalar()
                        new_volume = current_volume + (angle_diff * 0.006)
                        new_volume = np.clip(new_volume, 0.0, 1.0)
                        volume.SetMasterVolumeLevelScalar(new_volume, None)
                        print(f"Volume: {int(new_volume * 100)}%")
                        volume_control = True
                        last_volume_control_time = time.time()

                prev_left_angle = smooth_angle

            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    if time.time() - last_brightness_update > 0.2:
        try:
            sbc.set_brightness(cached_brightness, display=0)
            print(f"Brightness: {cached_brightness:.1f}%")
        except:
            pass
        last_brightness_update = time.time()

    # --- Draw the Meters with Glow Effects ---

    # Brightness bar (Horizontal Yellow Glow)
    if brightness_control and (time.time() - last_brightness_control_time) < control_display_duration:
        brightness_percent = int(cached_brightness)
        bar_length = int(np.interp(brightness_percent, [0, 100], [0, 300]))
        
        # Glow Effect
        overlay = frame.copy()
        glow_color = (0, 255, 255)  # Bright Yellow
        cv2.rectangle(overlay, (50, 400), (50 + bar_length, 430), glow_color, -1)
        alpha = 0.4
        frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

        # Bar Frame
        cv2.rectangle(frame, (50, 400), (50 + 300, 430), (50, 50, 50), 3)
        cv2.putText(frame, f'Brightness: {brightness_percent}%', (60, 395),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, glow_color, 2)

    # Volume bar (Vertical Blue Glow)
    if volume_control and (time.time() - last_volume_control_time) < control_display_duration:
        volume_percent = int(current_volume * 100)
        bar_height = int(np.interp(volume_percent, [0, 100], [300, 0]))
        
        # Glow Effect
        overlay = frame.copy()
        glow_color = (255, 0, 0)  # Bright Blue
        cv2.rectangle(overlay, (500, 100 + bar_height), (530, 400), glow_color, -1)
        alpha = 0.4
        frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

        # Bar Frame
        cv2.rectangle(frame, (500, 100), (530, 400), (50, 50, 50), 3)
        cv2.putText(frame, f'Volume: {volume_percent}%', (460, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, glow_color, 2)

    # --- Status Text ---
    status_right = "ACTIVE" if right_active else "INACTIVE"
    status_left = "ACTIVE" if left_active else "INACTIVE"

    cv2.putText(frame, f"Right Hand: {status_right}", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0) if right_active else (0, 0, 255), 2)
    cv2.putText(frame, f"Left Hand: {status_left}", (10, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0) if left_active else (0, 0, 255), 2)

    cv2.imshow("Rotation Control with Spiderman Activation + Meters + Glow", frame)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Brightness: 42.0%
Right hand active: True
Brightness: 49.7%
Brightness: 48.8%
Brightness: 61.5%
Brightness: 93.9%
Brightness: 95.3%
Brightness: 94.4%
Brightness: 90.7%
Brightness: 87.4%
Brightness: 86.0%
Brightness: 85.5%
Brightness: 85.5%
Brightness: 84.1%
Brightness: 84.1%
Brightness: 84.1%
Brightness: 84.1%
Left hand active: True
Brightness: 84.1%
Volume: 77%
Brightness: 84.1%
Volume: 54%
Volume: 22%
Volume: 0%
Brightness: 84.1%
Volume: 0%
Volume: 1%
Volume: 2%
Brightness: 84.1%
Brightness: 84.1%
Volume: 3%
Volume: 5%
Volume: 8%
Volume: 10%
Brightness: 84.1%
Volume: 13%
Volume: 16%
Volume: 18%
Volume: 19%
Brightness: 84.1%
Brightness: 84.1%
Brightness: 84.5%
Volume: 18%
Volume: 16%
Volume: 15%
Brightness: 86.5%
Volume: 14%
Brightness: 

gesture-controlled PowerPoint slide navigation

In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time
import math

# — Mediapipe setup —
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.4,
    min_tracking_confidence=0.4
)
mp_draw = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)

# — Parameters & state —
TOGGLE_CD = 5.0      # Cooldown period (seconds) for toggling between start/stop
GESTURE_CD = 1.0     # Cooldown period (seconds) for next/prev gestures
HOLD_THRESH = 3.0    # Seconds until hold-rapid kicks in
RAPID_INT = 0.2      # Seconds between rapid slides
finger_tips = [8, 12, 16, 20]

last_toggle = 0
last_gesture_time = 0  # For next/prev gesture cooldown
fist_active = False
active = False
hold_start = None
in_hold = False

# — Helpers —
def is_fist(lm):
    # Check if the hand is in a fist position (fingers are curled)
    for i in [4, 8, 12, 16, 20]:  # Check if all fingers are curled
        if lm[i][1] < lm[i-2][1]:  # The finger tip should be below the second joint
            return False
    return True

def index_up(lm):
    # Checks if only the index finger is up
    return lm[8][1] < lm[6][1] and all(lm[t][1] > lm[t-2][1] for t in [12, 16, 20])

def prev_up(lm):
    # Checks if both index and middle fingers are up
    return lm[8][1] < lm[6][1] and lm[12][1] < lm[10][1] and all(lm[t][1] > lm[t-2][1] for t in [16, 20])

# — Display a status bar at the top-right —
def show_status_bar(img, status_text):
    h, w, _ = img.shape
    bar_width = 200
    bar_height = 50
    color = (0, 255, 0) if status_text == "STARTED" else (0, 0, 255)

    # Draw the status bar
    cv2.rectangle(img, (w - bar_width - 10, 10), (w - 10, bar_height), color, -1)
    cv2.putText(img, status_text, (w - bar_width + 10, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# — Main loop —
while True:
    ret, img = cap.read()
    if not ret:
        break

    img = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    now = time.time()

    # Collect landmarks of each hand
    hands_lm = []
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            h, w, _ = img.shape
            lm = [(int(p.x * w), int(p.y * h)) for p in handLms.landmark]
            hands_lm.append((lm, handLms))

    # — Draw all detected hands —
    for _, handLms in hands_lm:
        mp_draw.draw_landmarks(img, handLms, mp_hands.HAND_CONNECTIONS)

    # — FIST GESTURE TOGGLE (edge-detect + cooldown) —
    if len(hands_lm) == 1:  # Only consider toggling if one hand is detected
        lm0, _ = hands_lm[0]
        if is_fist(lm0) and not fist_active and (now - last_toggle) > TOGGLE_CD:
            active = not active
            last_toggle = now
            fist_active = True
        if not is_fist(lm0):
            fist_active = False

    # — SLIDE CONTROLS (only when active) —
    if active and hands_lm:
        lm, handLms = hands_lm[0]

        # Check index up (next slide)
        if index_up(lm) and (now - last_gesture_time) > GESTURE_CD:
            pyautogui.press('right')
            last_gesture_time = now

        # Check prev up (previous slide)
        if prev_up(lm) and (now - last_gesture_time) > GESTURE_CD:
            pyautogui.press('left')
            last_gesture_time = now

    # Show status bar (STARTED or STOPPED)
    show_status_bar(img, "STARTED" if active else "STOPPED")

    cv2.imshow("Hand-Gesture Slide Control", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
